In [30]:
import dask.dataframe as dd
import polars as pl

In [31]:
lista = ['CTU-Honeypot-Capture-4-1','CTU-Honeypot-Capture-5-1','CTU-Honeypot-Capture-7-1', #2
         'CTU-IoT-Malware-Capture-1-1','CTU-IoT-Malware-Capture-3-1','CTU-IoT-Malware-Capture-7-1',#5
         'CTU-IoT-Malware-Capture-8-1','CTU-IoT-Malware-Capture-9-1','CTU-IoT-Malware-Capture-20-1',#8
         'CTU-IoT-Malware-Capture-21-1','CTU-IoT-Malware-Capture-34-1','CTU-IoT-Malware-Capture-35-1',#11
         'CTU-IoT-Malware-Capture-36-1','CTU-IoT-Malware-Capture-42-1','CTU-IoT-Malware-Capture-44-1', #14
         'CTU-IoT-Malware-Capture-48-1','CTU-IoT-Malware-Capture-49-1','CTU-IoT-Malware-Capture-60-1',] #17

caminhos_arquivos = [f'../../{nome_arquivo}.parquet' for nome_arquivo in lista]

In [32]:
dfs = [dd.read_parquet(caminho, engine='pyarrow') for caminho in caminhos_arquivos]
df = dd.concat(dfs)

In [33]:
#df = df.sample(frac=0.01, random_state=42)
del lista, dfs,caminhos_arquivos

# Pré-processamento

In [34]:
# Função para substituir "-" por "0" em uma coluna
def replace_dash_with_zero_dask(df, column):
    df[column] = df[column].replace("-", "0", regex=True)
    return df

# Aplicar a substituição nas colunas desejadas
df = df.map_partitions(replace_dash_with_zero_dask, column='duration')
df = df.map_partitions(replace_dash_with_zero_dask, column='orig_bytes')
df = df.map_partitions(replace_dash_with_zero_dask, column='resp_bytes')

# Conversão de tipos
df['ts'] = df['ts'].astype(float)
df['uid'] = df['uid'].astype(str)
df['id.orig_h'] = df['id.orig_h'].astype(str)
df['id.orig_p'] = df['id.orig_p'].astype(int)
df['id.resp_h'] = df['id.resp_h'].astype(str)
df['id.resp_p'] = df['id.resp_p'].astype(int)
df['proto'] = df['proto'].astype(str)
df['service'] = df['service'].astype(str)
df['duration'] = df['duration'].astype(float)
df['orig_bytes'] = df['orig_bytes'].astype(float)
df['resp_bytes'] = df['resp_bytes'].astype(float)
df['conn_state'] = df['conn_state'].astype(str)
df['local_orig'] = df['local_orig'].astype(str)
df['local_resp'] = df['local_resp'].astype(str)
df['missed_bytes'] = df['missed_bytes'].astype(int)
df['history'] = df['history'].astype(str)
df['tunnel_parents'] = df['tunnel_parents'].astype(str)
df['label'] = df['label'].astype(str)

In [ ]:
# Mapear labels
df['label'] = df['label'].str.replace('benign', 'Benign')
df['label'] = df['label'].map({'Benign': 0, 'Malicious': 1})
df['label'] = df['label'].astype(int)

In [36]:
label_columns = ['proto', 'service', 'conn_state', 'history']

def label_encode_column(df, col):
    unique_values = df[col].dropna().unique()
    label_mapping = {value: i for i, value in enumerate(unique_values)}
    df[col] = df[col].map(label_mapping)
    return df

for col in label_columns:
    df = df.map_partitions(label_encode_column, col=col)

In [ ]:
pandas_df = df.compute()

In [ ]:
df_polars = pl.from_pandas(pandas_df)

del pandas_df

In [ ]:
df_polars.write_parquet('dataset.parquet', compression='snappy')
print(f"Arquivo foi salvo com sucesso.")

In [ ]:
valores_unicos.head(7)

In [ ]:
valores_unicos.tail(7)

In [11]:
#arquivo = "dataset"
#df_pandas.to_parquet(f'{arquivo}.parquet', compression='snappy')

#print(f"Arquivo '{arquivo}.parquet' salvo com sucesso.")